# Combining Before and After Validators

We saw how to define before validators, and after validators.

We can define as many before and after validators as we want, and we can also mix both before and after validators.

The before validators will be executed first (bottom to top order), then the Pydantic validator, then the after validators (top to bottom).

Using both before and after validators can be quite handy.

Think back to our datetime example:

In [1]:
from datetime import datetime
from typing import Any

import pytz
from dateutil.parser import parse
from pydantic import BaseModel, field_validator, ValidationError


class Model(BaseModel):
    dt: datetime

    @field_validator("dt", mode="before")
    @classmethod
    def parse_datetime(cls, value: Any):
        if isinstance(value, str):
            try:
                return parse(value)
            except Exception as ex:
                raise ValueError(str(ex))
        return value

Here, we have a single validator that basically adds some customized parsing for strings, and leaves everything else as-is, leaving it to Pydantic to handle other types.

In fact, Pydantic can also handle integer values - it will just treat it as epoch:

In [2]:
Model(dt=100_000)

Model(dt=datetime.datetime(1970, 1, 2, 3, 46, 40, tzinfo=TzInfo(UTC)))

Now, if we look at what happens when we use a string:

In [3]:
Model(dt="2020/1/1 3pm")

Model(dt=datetime.datetime(2020, 1, 1, 15, 0))

You can see that we have a naive datetime, whereas with the epoch, we ended up wit an aware datetime.

There's a definite lack on consistency.

Let's suppose we want to also add in some code that will convert all datetimes (naive or aware) into aware UTC times (like the example we saw in the `After Validators` video.

We could certainly try and handle everything in the before validator - but that means we would have to handle all the various data types we ewant to support.

Instead, we'll handle just the strings in the before validator, let Pydantic do its thing, then use an after validator (now that we are guaranteed the value is a datetime object) to transform our datetime object into a UTC aware datetime.

Recall the `make_utc` validator we used:

In [4]:
class Model(BaseModel):
    dt: datetime

    @field_validator("dt")
    @classmethod
    def make_utc(cls, dt: datetime) -> datetime:
        if dt.tzinfo is None:
            dt = pytz.utc.localize(dt)
        else:
            dt = dt.astimezone(pytz.utc)
        return dt

So now, we are going to combine both of those validators:

In [5]:
class Model(BaseModel):
    dt: datetime

    @field_validator("dt", mode="before")
    @classmethod
    def parse_datetime(cls, value: Any):
        if isinstance(value, str):
            try:
                return parse(value)
            except Exception as ex:
                raise ValueError(str(ex))
        return value

    @field_validator("dt")
    @classmethod
    def make_utc(cls, dt: datetime) -> datetime:
        if dt.tzinfo is None:
            dt = pytz.utc.localize(dt)
        else:
            dt = dt.astimezone(pytz.utc)
        return dt

This now implements exactly what we were looking for:

In [6]:
Model(dt=100_000)

Model(dt=datetime.datetime(1970, 1, 2, 3, 46, 40, tzinfo=<UTC>))

In [7]:
Model(dt="2020/1/1 3pm")

Model(dt=datetime.datetime(2020, 1, 1, 15, 0, tzinfo=<UTC>))

In [8]:
eastern = pytz.timezone('US/Eastern')

Model(dt=eastern.localize(datetime(2020, 1, 1, 3, 0, 0)))

Model(dt=datetime.datetime(2020, 1, 1, 8, 0, tzinfo=<UTC>))

Lastly, before we finish up this video, I just want to illustrate what we discussed in the lecture video about the execution order when we have multiple mixed before and after validators.

In [9]:
class Model(BaseModel):
    number: int

    @field_validator("number")
    @classmethod
    def after_validator_1(cls, value):
        print("after_validator_1")
        return value
        
    @field_validator("number")
    @classmethod
    def after_validator_2(cls, value):
        print("after_validator_2")
        return value
        
    @field_validator("number", mode="before")
    @classmethod
    def before_validator_1(cls, value):
        print("before_validator_1")
        return value
        
    @field_validator("number")
    @classmethod
    def after_validator_3(cls, value):
        print("after_validator_3")
        return value

    @field_validator("number", mode="before")
    @classmethod
    def before_validator_2(cls, value):
        print("before_validator_2")
        return value

In [10]:
Model(number=10)

before_validator_2
before_validator_1
after_validator_1
after_validator_2
after_validator_3


Model(number=10)